In [1]:
import pandas as pd
import scanpy as sc

In [2]:
#已有的gene和peak之间的关系
df=pd.read_csv('./data/peak2gene_peaks_genesplit_filtered.csv')

In [3]:
#合并过来的h5ad文件
adata=sc.read_h5ad('/home/share/huadjyin/home/caixianjun/Projects/scRNA/demo/HIV_temp/results/HIV_35_neighbors_final_matched.h5ad',backed='r')

In [4]:
#获得h5ad的基因和peak的列表
list2=adata.var.index.tolist()
list_gene=list2[:591]
list_peak=list2[591:]

In [5]:
#部分h5ad的基因在df表格里不存在关系，需要删除
list_gene_all=df['Matched_Gene'].unique().tolist()

In [6]:
import pickle
def save_data(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

# 定义一个函数，用于加载文件中的数据
# 定义一个函数，用于加载文件中的数据
    # 打开文件，以二进制模式读取
def load_data(filename):
        # 使用pickle模块加载文件中的数据
    with open(filename, 'rb') as f:
    # 返回加载的数据
        data = pickle.load(f)
    return data
save_data(list_gene_all, './data/list_gene_all.pkl')
save_data(list_peak, './data/list_peak.pkl')

Data saved to ./data/list_gene_all.pkl
Data saved to ./data/list_peak.pkl


In [11]:
adata2=adata[:,list_gene_all+list_peak]

In [ ]:
adata2.write_h5ad('./data/adata_process.h5ad')

In [6]:
dict_gene={gene:[0 for i in range(len(list_peak))] for gene in list_gene_all}

In [7]:
from tqdm import tqdm
for index, row in tqdm(df.iterrows()):
    gene=row['Matched_Gene']
    peak=row['position']
    dict_gene[gene][list_peak.index(peak)]=1


7200it [00:01, 6586.02it/s]


In [8]:
list_all=[]
for gene in list_gene_all:
    list_all.append(dict_gene[gene])

In [ ]:
import numpy as np
array_mat=np.array(list_all)


In [ ]:
import torch
tensor_mask_gene=torch.tensor(array_mat,dtype=torch.float32)
torch.save(tensor_mask_gene,'./data/mask_mat.pt')